<a href="https://colab.research.google.com/github/victor-roris/mediumseries/blob/master/NLP/ModelInterpretability_ELI5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ELI5 - Explain Like I am 5

ELI5 provides both global model interpretation and local model interpretation. 
The global interpretation works for: scikit-learn, Keras, xgboost, LightGBM, CatBoost, lightning, sklearn-crfsuite. 
For local model interpretation, ELI5 use LIME algorithm.  The display format is different from LIME but using same idea. 

DOCUMENTATION: https://eli5.readthedocs.io/en/latest/index.html

GITHUB: https://github.com/TeamHG-Memex/eli5

ADAPTED FROM: https://github.com/makcedward/nlp/blob/master/sample/nlp-model_interpretation.ipynb


## INSTALLATION

In [0]:
!pip install eli5

## EXAMPLE OF USAGE

In [14]:
import random
import pandas as pd
import numpy as np 
import IPython
import xgboost
import keras

import eli5
print('ELI5 Version:', eli5.__version__)
print('XGBoost Version:', xgboost.__version__)
print('Keras Version:', keras.__version__)

ELI5 Version: 0.10.1
XGBoost Version: 0.90
Keras Version: 2.2.5


### Train NLP models

We generate a set of NLP models to test the ELI5 library

#### Fetch data

We use the sklearn direct dataset 20 news groups.

In [4]:
from sklearn.datasets import fetch_20newsgroups
train_raw_df = fetch_20newsgroups(subset='train')
test_raw_df = fetch_20newsgroups(subset='test')

In [8]:
print(f'Number of raw training examples: {len(train_raw_df.data)}')
print(f'Number of raw test examples: {len(test_raw_df.data)}')

Number of raw training examples: 11314
Number of raw test examples: 7532


In [17]:
category_names = np.unique(np.array(train_raw_df.target_names))
print(f'Number of different categories : {len(category_names)}')
print(f'Category list: {category_names}')

Number of different categories : 20
Category list: ['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


In [11]:
print('Example of entry:')
print(f'\t - LABEL : {train_raw_df.target[0]} - {train_raw_df.target_names[0]}')
print(f'\t - {train_raw_df.data[0]}')

Example of entry:
	 - LABEL : 7 - alt.atheism
	 - From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







#### Prepare data to the model

In [0]:
x_train = train_raw_df.data
y_train = train_raw_df.target

x_test = test_raw_df.data
y_test = test_raw_df.target

#### Training models

* **Models definition**

We are going to use 4 different models: Logistic Regression, Random Forest, XGBoost and a Keras model.

In [0]:
# Word Embedding via TFIDF - vectorization of the text
from sklearn.feature_extraction.text import TfidfVectorizer

# Pipeline execution to combine vectorization and model execution
from sklearn.pipeline import make_pipeline 

# Definition of generic models
from sklearn.linear_model import LogisticRegression  # Logistic Regression
from sklearn.ensemble import RandomForestClassifier  # Random Forest
from xgboost import XGBClassifier                    # XGBoost

# Definition of a custom model in Keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin

from keras.models import Model, Input
from keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Bidirectional, concatenate
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

class KerasTextClassifier:
    __author__ = "Edward Ma"
    __copyright__ = "Copyright 2018, Edward Ma"
    __credits__ = ["Edward Ma"]
    __license__ = "Apache"
    __version__ = "2.0"
    __maintainer__ = "Edward Ma"
    __email__ = "makcedward@gmail.com"
    
    OOV_TOKEN = "UnknownUnknown"
    
    def __init__(self, 
                 max_word_input, word_cnt, word_embedding_dimension, labels, 
                 batch_size, epoch, validation_split,
                 verbose=0):
        self.verbose = verbose
        self.max_word_input = max_word_input
        self.word_cnt = word_cnt
        self.word_embedding_dimension = word_embedding_dimension
        self.labels = labels
        self.batch_size = batch_size
        self.epoch = epoch
        self.validation_split = validation_split
        
        self.label_encoder = None
        self.classes_ = None
        self.tokenizer = None
        
        self.model = self._init_model()
        self._init_label_encoder(y=labels)
        self._init_tokenizer()
        
    def _init_model(self):
        input_layer = Input((self.max_word_input,))
        text_embedding = Embedding(
            input_dim=self.word_cnt+2, output_dim=self.word_embedding_dimension,
            input_length=self.max_word_input, mask_zero=False)(input_layer)
        
        text_embedding = SpatialDropout1D(0.5)(text_embedding)
        
        bilstm = Bidirectional(LSTM(units=256, return_sequences=True, recurrent_dropout=0.5))(text_embedding)
        x = concatenate([GlobalAveragePooling1D()(bilstm), GlobalMaxPooling1D()(bilstm)])
        x = Dropout(0.5)(x)
        x = Dense(128, activation="relu")(x)
        x = Dropout(0.5)(x)
        
        output_layer = Dense(units=len(self.labels), activation="softmax")(x)
        model = Model(input_layer, output_layer)
        model.compile(
            optimizer="adam",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model
    
    def _init_tokenizer(self):
        self.tokenizer = Tokenizer(
            num_words=self.word_cnt+1, split=' ', oov_token=self.OOV_TOKEN)
    
    def _init_label_encoder(self, y):
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(y)
        self.classes_ = self.label_encoder.classes_
        
    def _encode_label(self, y):
        return self.label_encoder.transform(y)
        
    def _decode_label(self, y):
        return self.label_encoder.inverse_transform(y)
    
    def _get_sequences(self, texts):
        seqs = self.tokenizer.texts_to_sequences(texts)
        return pad_sequences(seqs, maxlen=self.max_word_input, value=0)
    
    def _preprocess(self, texts):
        # Placeholder only.
        return [text for text in texts]
        
    def _encode_feature(self, x):
        self.tokenizer.fit_on_texts(self._preprocess(x))
        self.tokenizer.word_index = {e: i for e,i in self.tokenizer.word_index.items() if i <= self.word_cnt}
        self.tokenizer.word_index[self.tokenizer.oov_token] = self.word_cnt + 1
        return self._get_sequences(self._preprocess(x))
        
    def fit(self, X, y):
        """
            Train the model by providing x as feature, y as label
        
            :params x: List of sentence
            :params y: List of label
        """
        
        encoded_x = self._encode_feature(X)
        encoded_y = self._encode_label(y)
        
        self.model.fit(encoded_x, encoded_y, 
                       batch_size=self.batch_size, epochs=self.epoch, 
                       validation_split=self.validation_split)
        
    def predict_proba(self, X, y=None):
        encoded_x = self._get_sequences(self._preprocess(X))
        return self.model.predict(encoded_x)
    
    def predict(self, X, y=None):
        y_pred = np.argmax(self.predict_proba(X), axis=1)
        return self._decode_label(y_pred)

In [0]:
model_names = ['Logistic Regression', 'Random Forest', 'XGBoost Classifier', 'Keras']

* **Build models**

We are going to create an object for each of the models.

In [0]:
def build_model(names, x, y):
    pipelines = []
    vec = TfidfVectorizer()
    vec.fit(x)

    for name in names:
        print('train %s' % name)
        
        if name == 'Logistic Regression':
            estimator = LogisticRegression(solver='newton-cg', n_jobs=-1)
            pipeline = make_pipeline(vec, estimator)

        elif name == 'Random Forest':
            estimator = RandomForestClassifier(n_jobs=-1)
            pipeline = make_pipeline(vec, estimator)

        elif name == 'XGBoost Classifier':
            estimator = XGBClassifier()
            pipeline = make_pipeline(vec, estimator)
            
        elif name == 'Keras':
            pipeline = KerasTextClassifier(
                max_word_input=100, word_cnt=30000, word_embedding_dimension=100, 
                labels=list(set(y_train.tolist())), batch_size=128, epoch=1, validation_split=0.1)
        
        
        pipeline.fit(x, y)
        pipelines.append({
            'name': name,
            'pipeline': pipeline
        })
        
    return pipelines, vec

In [26]:
pipelines, vec = build_model(model_names, x_train, y_train)

train Logistic Regression


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


train Random Forest


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train XGBoost Classifier
train Keras




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 10182 samples, validate on 1132 samples
Epoch 1/1





10182/10182 [==============================] - 222s 22ms/step - loss: 2.9675 - acc: 0.0668 - val_loss: 2.8538 - val_acc: 0.0972


### ELI5 - Model interpretability

ELI5 provides both global model interpretation and local model interpretation. 





#### Global Interpretation

You may simply consider the global model interpretation as a feature importance but it not **only support decision tree algorithm** such as Radom Forest and XGBoost but also all sci-kit learn estimators.

ELI5 author introduces it as **Permutation Importance** for global interpretation. To calculate the score, feature (a word) will be replaced by other word (noise) and predicting it. The idea is that feature importance can be deduced by getting how much score decrease when a particular word is not provided. For example, "I like apple". It will may changed to "I like orange" and then it will classify the newly created record to understand how "apple" is important. Of course, we need to assume the replaced word (e.g. orange) is noise and it should not provide major change on score.


In [34]:
for pipeline in pipelines:
  
    print()
    print('Estimator: %s' % (pipeline['name']))
    # labels = pipeline['pipeline'].classes_.tolist()
    labels = category_names
    
    if pipeline['name'] in ['Logistic Regression', 'Random Forest']:
        estimator = pipeline['pipeline']

    elif pipeline['name'] == 'XGBoost Classifier':
        estimator = pipeline['pipeline'].steps[1][1].get_booster()
    
    # Not support Keras
    elif pipeline['name'] == 'Keras':
        # estimator = pipeline['pipeline']
        print('\t - Global Interpretation is not supported to Keras')
        continue

    else:
        continue
    
    IPython.display.display(
        eli5.show_weights(estimator=estimator, top=10, target_names=labels, vec=vec))


Estimator: Logistic Regression



Estimator: Random Forest


Weight,Feature
0.0060 ± 0.0093,space
0.0059 ± 0.0121,baseball
0.0056 ± 0.0089,sale
0.0045 ± 0.0046,windows
0.0042 ± 0.0047,of
0.0039 ± 0.0094,bike
0.0035 ± 0.0080,graphics
0.0034 ± 0.0077,chip
0.0034 ± 0.0132,encryption
0.0032 ± 0.0080,israel



Estimator: XGBoost Classifier


Weight,Feature
0.0294,geb
0.0185,msg
0.0116,dod
0.0094,clipper
0.0090,cramer
0.0083,sale
0.0079,hendricks
0.0071,israel
0.0070,hockey
0.0063,armenia



Estimator: Keras
	 - Global Interpretation is not supported to Keras


The above figure means that, if input includes "keith", then score of y=0 (atheism) increase 4.858. Another case is "the", it will decrease score -5.674 in y=6 (forsale)

#### Local Interpretation

For local model interpretation, ELI5 use LIME algorithm. The display format is different from LIME but using same idea.


* **Local Interpretation using show_prediction method**

ELI5 supports `eli5.show_prediction()` to conveniently invoke `explain_prediction` with format_as_image, and display the explanation in an `IPython cell`.

In [42]:
number_of_sample = 1
sample_ids = [random.randint(0, len(x_test) -1 ) for p in range(0, number_of_sample)]

for idx in sample_ids:
    print('Index: %d' % (idx))
    
    for pipeline in pipelines:
        print('-' * 50)
        print('Estimator: %s' % (pipeline['name']))
        
        print('True Label: %s, Predicted Label: %s' % (y_test[idx], pipeline['pipeline'].predict([x_test[idx]])[0]))
        # labels = pipeline['pipeline'].classes_.tolist()
        labels = category_names
  
        if pipeline['name'] in ['Logistic Regression', 'Random Forest']:
            estimator = pipeline['pipeline'].steps[1][1]
        elif pipeline['name'] == 'XGBoost Classifier':
            estimator = pipeline['pipeline'].steps[1][1].get_booster()
        # Not support Keras
        elif pipeline['name'] == 'Keras':
            # estimator = pipeline['pipeline'].model
            print('\t - Local Interpretation, "show_prediction" method doesnt support Keras')
            # continue
        else:
            continue

        IPython.display.display(
            eli5.show_prediction(estimator, x_test[idx], top=10, vec=vec, target_names=labels))

Index: 5364
--------------------------------------------------
Estimator: Logistic Regression
True Label: 17, Predicted Label: 17


--------------------------------------------------
Estimator: Random Forest
True Label: 17, Predicted Label: 17


--------------------------------------------------
Estimator: XGBoost Classifier
True Label: 17, Predicted Label: 17


--------------------------------------------------
Estimator: Keras
True Label: 17, Predicted Label: 17
	 - Local Interpretation is not supported to Keras


* **Local prediction using Lime TextExplainer**

https://eli5.readthedocs.io/en/latest/tutorials/black-box-text-classifiers.html#lime-tutorial

We use this method to explain Keras model.

In [59]:
import eli5
from eli5.lime import TextExplainer


target_names = pipeline['pipeline'].classes_.tolist()
te = TextExplainer(random_state=42)
te.fit(x_test[idx], pipeline['pipeline'].predict_proba)
te.show_prediction(target_names=target_names)